In [ ]:
import pickle
import sys
from zoneinfo import ZoneInfo
sys.path.append("../")
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()
import geopandas as gpd
import importlib
import copy
import logging
import contextily as cx
import gtfs_kit as gk
import fastsim as fsim
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import lightning.pytorch as pl
import rasterio as rio
import seaborn as sns
from rasterio.plot import show
import seaborn as sns
from sklearn.cluster import KMeans
import shapely
import statsmodels.api as sm
from torch.utils.data import DataLoader

from openbustools.traveltime.models import rnn
from openbustools import plotting, spatial, standardfeeds
from openbustools.traveltime import data_loader, model_utils
from openbustools.drivecycle import trajectory, busnetwork
from openbustools.drivecycle.physics import conditions, energy, vehicle

### Energy Use for KCM Network

In [ ]:
network_name = "kcm"
res_dir = Path("..","results","energy", network_name)
epsg = 32148

file = open(res_dir / "trajectories_updated.pkl", "rb")
trajectories_updated = pickle.load(file)
file.close()

file = open(res_dir / "depot_locations.pkl", "rb")
depot_locations = pickle.load(file)
file.close()

file = open(res_dir / "cycles.pkl", "rb")
cycles = pickle.load(file)
file.close()

file = open(res_dir / "network_energy.pkl", "rb")
network_energy = pickle.load(file)
file.close()

file = open(res_dir / "network_charging.pkl", "rb")
network_charging = pickle.load(file)
file.close()

file = open(res_dir / "veh_status.pkl", "rb")
veh_status = pickle.load(file)
file.close()

In [ ]:
# Report table
block_energy_summary = network_energy.groupby('block_id').first()[['block_dist_mi', 'block_consumption_kwh_mi', 'block_net_energy_kwh']]
block_energy_summary.describe().transpose()

In [ ]:
# Drive cycle reported trip consumption
fig, axes = plt.subplots(1,1, figsize=(6,5))
sns.histplot(block_energy_summary['block_consumption_kwh_mi'], kde=True, ax=axes)
axes.axvline(x=np.mean(block_energy_summary['block_consumption_kwh_mi']), linestyle='dashed', color='r')
axes.set_xlabel("Consumption (kWh/mi)")
fig.suptitle(f"BEB Block Consumption Distribution\n{len(block_energy_summary)} Blocks - (Avg. {np.mean(block_energy_summary['block_consumption_kwh_mi']):.2} kWh/mi)")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_block_consumption_distribution.png"))
plt.show()

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(6,5))
sns.ecdfplot(block_energy_summary['block_net_energy_kwh'], ax=axes)
axes.axvline(x=466, linestyle='dashed', color='r')
axes.text(475, 0.4, "New Flyer XE40", color='black', fontsize=12)
axes.set_xlabel("Block Required Energy (kWh)")
axes.set_ylabel("Proportion of Total Blocks")
fig.suptitle(f"BEB Block Energy Distribution\n{len(block_energy_summary)} Blocks - (Avg. {np.mean(block_energy_summary['block_net_energy_kwh']):.0f} kWh)")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_block_energy_distribution.png"))
plt.show()

In [ ]:
# Depot locations
block_starts_df = network_energy.groupby('block_id').first()
block_starts_df = gpd.GeoDataFrame(block_starts_df, geometry=gpd.points_from_xy([t.x for t in block_starts_df['start_loc']], [t.y for t in block_starts_df['start_loc']])).set_crs(epsg)
depot_df = gpd.GeoDataFrame(depot_locations, geometry=gpd.points_from_xy(depot_locations['depot_x'], depot_locations['depot_y'])).set_crs(epsg)

fig, axes = plt.subplots(1,1, figsize=(6,6))
block_starts_df.plot(ax=axes, column='depot_id', markersize=10, cmap='tab20')
depot_df.plot(ax=axes, marker='x', markersize=100, color="blue", linewidth=3)
cx.add_basemap(ax=axes, crs=block_starts_df.crs.to_string(), source=cx.providers.CartoDB.Positron)
axes.set_xticks([])
axes.set_yticks([])
fig.suptitle("Clustered Depot Locations")
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_depot_locations.png"))

### Validate Block Energy w/KCM Report

In [ ]:
# Load the daily summaries from KCM report
summary_data = standardfeeds.clean_parametrix("../data/bebdatafollowup/Viriciti_Energy_Reports-2023.csv")
summary_data = summary_data[summary_data['DateTime'] >= datetime(2023, 12, 1)]
summary_data['realtime_filename'] = summary_data['DateTime'].dt.strftime("%Y_%m_%d")
summary_data = summary_data.groupby(['realtime_filename','vehicle_id','metric']).agg({'value':'mean'}).reset_index().sort_values(['realtime_filename', 'vehicle_id', 'metric'])

# Load the most recent static feed
static_path = Path("..","data","kcm_static","2023_09_27")
static = gk.read_feed(static_path, dist_units='km')

# Load realtime data from all BEB vehicle IDs in the KCM report
realtime_path = Path("..","data","kcm_realtime","processed", "analysis")
beb_ids = summary_data['vehicle_id'].unique()
beb_dates = summary_data['realtime_filename'].unique()
all_realtime_data = []
for d in beb_dates:
    realtime_data = pd.read_pickle(Path(f"../data/kcm_realtime/processed/analysis/{d}.pkl"))
    realtime_data = realtime_data[realtime_data['vehicle_id'].isin(beb_ids)]
    realtime_data['realtime_filename'] = realtime_data['realtime_filename'].str[:-4]
    all_realtime_data.append(realtime_data)
all_realtime_data = pd.concat(all_realtime_data).sort_values(['realtime_filename','vehicle_id','trip_id','locationtime'])

# Map (day, vehicle_id) > trip_ids using the realtime data
trip_id_lookup = all_realtime_data[['realtime_filename','vehicle_id','trip_id']].drop_duplicates().copy()
# Map trip_id > (service_id, block_id) using the static data
block_id_lookup = static.get_trips()[['service_id','block_id','trip_id']].drop_duplicates().copy()
block_id_lookup = pd.merge(trip_id_lookup, block_id_lookup, on='trip_id')
block_id_lookup = block_id_lookup[['realtime_filename','vehicle_id','service_id','block_id']].drop_duplicates().copy()
# Join energy summaries to their block_ids; note there are days where the vehicle was tracked on multiple blocks in the realtime
summary_data = pd.merge(summary_data, block_id_lookup, on=['realtime_filename','vehicle_id'])
# Get comparison metrics for each block
summary_data_means = summary_data.groupby(['block_id','metric'], as_index=False).agg({'value': 'mean'}).pivot(index='block_id', columns='metric', values='value')
summary_data_stds = summary_data.groupby(['block_id','metric'], as_index=False).agg({'value': 'std'}).pivot(index='block_id', columns='metric', values='value').mean()
real_beb_comparison = pd.merge(summary_data_means, block_energy_summary, on='block_id')
real_beb_comparison.head()

In [ ]:
plot_df = real_beb_comparison
plot_df['block_dist_60ft_mi'] = plot_df['block_dist_mi'] * 1.4
plot_df['block_consumption_60ft_kwh_mi'] = plot_df['block_consumption_kwh_mi'] * 1.5
plot_df['block_net_energy_60ft_kwh'] = plot_df['block_consumption_60ft_kwh_mi'] * plot_df['block_dist_60ft_mi']

fig, axes = plt.subplots(1,2, figsize=(12,6))
sns.scatterplot(data=plot_df, x='block_net_energy_kwh', y='Energy used', ax=axes[0])
axes[0].axline([0,0], [1,1], color='red', linestyle='--', alpha=.8)
axes[0].axline([0, 0+summary_data_stds['Energy used']], [1, 1+summary_data_stds['Energy used']], color='red', alpha=.3)
axes[0].axline([0, 0-summary_data_stds['Energy used']], [1, 1-summary_data_stds['Energy used']], color='red', alpha=.3)
axes[0].set_xlim(0, 800)
axes[0].set_ylim(0, 800)
axes[0].set_xlabel("Modeled Energy (kWh)")
axes[0].set_ylabel("Actual Energy (kWh)")
axes[0].set_title("40ft Bus")

sns.scatterplot(data=plot_df, x='block_net_energy_60ft_kwh', y='Energy used', ax=axes[1])
axes[1].axline([0,0], [1,1], color='red', linestyle='--', alpha=.8)
axes[1].axline([0, 0+summary_data_stds['Energy used']], [1, 1+summary_data_stds['Energy used']], color='red', alpha=.3)
axes[1].axline([0, 0-summary_data_stds['Energy used']], [1, 1-summary_data_stds['Energy used']], color='red', alpha=.3)
axes[1].set_xlim(0, 800)
axes[1].set_ylim(0, 800)
axes[1].set_xlabel("Modeled Energy (kWh)")
axes[1].set_ylabel("Actual Energy (kWh)")
axes[1].set_title("Model Adjusted for 60ft Bus")
fig.suptitle(f"Modeled vs. Actual Block Energy Consumption for KCM\n{len(plot_df)} Blocks")
fig.tight_layout()
fig.savefig(Path("..", "plots", "kcm_real_beb_comparison.png"))
plt.show()

### Charging for KCM Network

In [ ]:
# Minimum charging rate to cover block energy
print(np.quantile(network_charging['min_charge_rate'], [.50, .90, .95, .98]))
print(network_charging['min_charge_rate_managed'].iloc[0])

In [ ]:
t_mins = np.arange(0, 1440+1440*2)
veh_status_df = pd.DataFrame({
    't_min_of_day': t_mins,
    'tot_veh_active': [len(network_charging[(network_charging['t_min_of_day']<=t) & (network_charging['t_min_of_day_end']>=t)]) for t in t_mins],
    'tot_veh_inactive': [len(network_charging[(network_charging['t_min_of_day']>t) | (network_charging['t_min_of_day_end']<t)]) for t in t_mins],
    'tot_veh_charging': [len(network_charging[(network_charging['t_charge_start_min']<=t) & (network_charging['t_charge_end_min']>=t)]) for t in t_mins],
    'tot_veh_arriving': [len(network_charging[network_charging['t_min_of_day_end']==t]) for t in t_mins],
    'tot_veh_departing': [len(network_charging[network_charging['t_min_of_day']==t]) for t in t_mins],
    'tot_energy_arriving': [network_charging[network_charging['t_min_of_day_end']==t]['block_net_energy_kwh'].sum() for t in t_mins],
    'tot_energy_departing': [network_charging[network_charging['t_min_of_day']==t]['block_net_energy_kwh'].sum() for t in t_mins],
    'tot_power': [network_charging[(network_charging['t_charge_start_min']<=t) & (network_charging['t_charge_end_min']>=t)]['plug_power_kw'].sum() for t in t_mins],
})
# Reset time to 0-1440
veh_status_df.loc[veh_status_df['t_min_of_day'] >= 2*1440, 't_min_of_day'] -= 2*1440
veh_status_df.loc[veh_status_df['t_min_of_day'] >= 1440, 't_min_of_day'] -= 1440
veh_status_df = veh_status_df.groupby('t_min_of_day', as_index=False).agg({
    'tot_veh_active': 'sum',
    'tot_veh_inactive': 'min',
    'tot_veh_charging': 'sum',
    'tot_veh_arriving': 'sum',
    'tot_veh_departing': 'sum',
    'tot_energy_arriving': 'sum',
    'tot_energy_departing': 'sum',
    'tot_power': 'sum'
}).sort_values('t_min_of_day')
veh_status_df

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(10,5))
axes2 = plt.twinx()

sns.lineplot(veh_status_df, x='t_min_of_day', y='tot_veh_active', ax=axes, color=sns.color_palette()[0], label="Active")
sns.lineplot(veh_status_df, x='t_min_of_day', y='tot_veh_inactive', ax=axes, color=sns.color_palette()[1], label="Inactive")
axes.set_ylim(0,1500)
axes.set_xlabel("Time of Day (minutes)")
axes.set_ylabel("Number of Vehicles")
axes.legend().remove()

sns.lineplot(veh_status_df, x='t_min_of_day', y='tot_power', ax=axes2, color=sns.color_palette()[2], label="Power Unmanaged")
axes2.axhline(y=veh_status_df['tot_power'].max(), linestyle='dashed', color=sns.color_palette()[2])
axes2.text(160, veh_status_df['tot_power'].max()-.05*veh_status_df['tot_power'].max(), f"Peak Power Demand: {veh_status_df['tot_power'].max()/1000:.1f} MW", color='black', fontsize=12)
axes2.set_xlabel("Time of Day (minutes)")
axes2.set_ylabel("Power Usage (kW)")
axes2.legend().remove()

fig.suptitle("Vehicle Status by Time of Day")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","kcm_veh_status.png"))

In [ ]:
min_bins = np.arange(0, 1440, 15)
veh_status_df['t_min_of_day_bin'] = np.digitize(veh_status_df['t_min_of_day'], min_bins) * 15
plot_df = veh_status_df.groupby('t_min_of_day_bin').sum()

fig, axes = plt.subplots(1,1, figsize=(10,5))
axes2 = plt.twinx()

sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_veh_arriving', ax=axes, color=sns.color_palette()[0], label="Vehicles Arriving (15min)")
sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_veh_departing', ax=axes, color=sns.color_palette()[1], label="Vehicles Departing (15min)")
axes.set_xlabel("Time of Day (minutes)")
axes.set_ylabel("Number of Vehicles")

sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_energy_arriving', ax=axes2, color=sns.color_palette()[3], linestyle='dashed', label="Power Unmanaged")
sns.lineplot(plot_df, x='t_min_of_day_bin', y='tot_energy_departing', ax=axes2, color=sns.color_palette()[2], linestyle='dashed', label="Power Unmanaged")
axes2.set_xlabel("Time of Day (minutes)")
axes2.set_ylabel("Departing Energy Needs (kWh)")
axes2.legend().remove()

fig.suptitle("Vehicle Availability and Needs by Time of Day")
fig.tight_layout()
plt.show()
fig.savefig(Path("..","plots","kcm_block_pullout.png"))

### KCM Sensitivity Analysis and Performance Metrics

In [ ]:
# TODO: Debug read and summarize the sensitivity, run energy + sensitivity for other networks, make other network plots

In [ ]:
kcm_sensitivity = pd.read_pickle(Path("..","results","energy","kcm","network_sensitivity.pkl"))

In [ ]:
# Compare metrics across sensitivity parameters
fig, axes = plt.subplots(len(kcm_sensitivity['metric'].unique())//2, 2, figsize=(20,15))
axes = axes.flatten()
for i, metric in enumerate(kcm_sensitivity['metric'].unique()):
    sns.boxplot(kcm_sensitivity[kcm_sensitivity['metric']==metric], x='sensitivity_parameter', y='value', ax=axes[i])
    axes[i].set_title(metric)
    axes[i].set_xlabel("")
    axes[i].set_ylabel(metric)
    axes[i].set_title("")
    # Rotate x labels
    for tick in axes[i].get_xticklabels():
        tick.set_rotation(20)
fig.tight_layout()
plt.show()
fig.savefig(Path("..", "plots", "kcm_sensitivity.png"))

### All Networks Sensitivity Analysis and Performance Metrics

In [ ]:
cleaned_sources = pd.read_csv("../data/cleaned_sources.csv")

all_network_res = []
for i,row in cleaned_sources.iloc[:1].iterrows():
    try:
        network_sensitivity = pd.read_pickle(Path("..","results","energy",row['uuid'],"network_sensitivity.pkl"))
        all_network_res.append(network_sensitivity)
    except Exception as e:
        continue
all_network_res = pd.concat(all_network_res)

In [ ]:
# # Compare metrics across sensitivity parameters
# fig, axes = plt.subplots(len(all_network_res['metric'].unique()), 1, figsize=(10,30))
# for i, metric in enumerate(all_network_res['metric'].unique()):
#     sns.boxplot(all_network_res[all_network_res['metric']==metric], x='sensitivity_parameter', y='value', ax=axes[i])
#     axes[i].set_title(metric)
#     axes[i].set_xlabel("")
#     axes[i].set_ylabel(metric)
#     axes[i].set_title("")
#     # Rotate x labels
#     for tick in axes[i].get_xticklabels():
#         tick.set_rotation(20)
# fig.tight_layout()
# plt.show()